In [1]:
import numpy as np
import os
import librosa
import torch
import torchcrepe

In [2]:
for instrument in ['piano', 'flute', 'harpsichord', 'string']:
    print(instrument)
    record_idx = [x.replace('.wav', '') for x in os.listdir('../data/Music/wavs/' + instrument)]
    for record_id in record_idx:
        audio, _ = librosa.load('../data/Music/wavs/%s/%s.wav' % (instrument, record_id))
        audio = torch.from_numpy(audio).float().unsqueeze(0).cuda()
        pitch, _ = torchcrepe.predict(audio, sample_rate=22050, hop_length=256, fmin=50, fmax=2000, model='full', 
                                      decoder=torchcrepe.decode.weighted_argmax, return_periodicity=True, batch_size=2048, device= 'cuda')
        pitch_smoothed = torchcrepe.filter.mean(pitch, 11)[:, 2:-1].log()
        pitch_smoothed = torchcrepe.filter.mean(pitch_smoothed, 11).squeeze(0).detach().cpu().numpy()
        np.save('../data/Music/pitch/%s/%s.npy' % (instrument, record_id), pitch_smoothed)